In [234]:
import keras
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Activation
from keras.utils.vis_utils import plot_model
from IPython.display import clear_output

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets, linear_model, preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline


In [235]:
features = ['MIP', 'SIP','EKIP','SKIP','MDM-SR','SDM-SR','EKDM-SR','SKDM-SR','Target']
df =  pd.read_csv('pulsar_stars.csv',
                names=features,na_values='?',header=None, skiprows = 1)
df = df.dropna()
df.head(6)

,MIP,SIP,EKIP,SKIP,MDM-SR,SDM-SR,EKDM-SR,SKDM-SR,Target
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
5,93.570312,46.698114,0.531905,0.416721,1.636288,14.545074,10.621748,131.394004,0


In [236]:

#pulsar = np.array((df.loc[:,'MIP':'Target']))
# Get the appropriate data from the datafile using the column headers
pulsar_x= np.array((df.loc[:,'MIP':'SKDM-SR']))
pulsar_y= np.array((df.loc[:,'Target']))

# Reshape the features and target vector
pulsar_x=pulsar_x.reshape(pulsar_x.shape[0],8)
pulsar_y=pulsar_y.reshape(pulsar_y.shape[0],1)

# Check the shape
print(pulsar_x.shape)
print(pulsar_y.shape)


(17898, 8)
(17898, 1)


In [237]:
# Scale the features, mean center and standard deviation = 1
x_scale = preprocessing.scale(pulsar_x)

# Default test size of 0.25 of dataset, default shuffle=true, 
x_train, x_test, y_train, y_test = train_test_split(x_scale, pulsar_y)
y_train = y_train.reshape((y_train.shape[0]), 1)

#Check the shape 
print(y_train.shape)
print(x_train.shape)

(13423, 1)
(13423, 8)


# Neural Networks
In the following section, we will run our data set through various neural networks with different hyper parameters. We will vary the activation function (tanh, reLU, sigmoid), the number of inputs in the hidden layer (10, 100, 200), and the number of hidden layers (1, 2, 5, with 100 nodes in each hidden layer). Weights and biases will be initialized randomly.  

### 1 Hidden Layer, 100 Hidden Nodes, Sigmoid Activation

In [238]:
K.clear_session()

# Number of nodes in input layer = number of features = 8
nin = x_train.shape[1];
# Number of hidden layer nodes
nh = 100;
# Number of output nodes
nout = 1;

# We wil use sigmoid as our activation here with 100 units in the hidden layer
model_sig = Sequential();
model_sig.add(Dense(nh, input_shape=(nin,), activation="sigmoid", name="hidden"))
model_sig.add(Dense(nout, activation="sigmoid", name="output"))

# We will use tanh as our activation here with 100 units in the hidden layer
model_tanh = Sequential();
model_tanh.add(Dense(nh, input_shape=(nin,), activation="tanh", name="hidden"))
model_tanh.add(Dense(nout, activation="sigmoid", name="output"))

# We will use reLU as out activation here with 100 units in the hidden layer
model_relu = Sequential();
model_relu.add(Dense(nh, input_shape=(nin,), activation="relu", name="hidden"))
model_relu.add(Dense(nout, activation="sigmoid", name="output"))


# We will use 10 hidden units in our network here with sigmoid as activation
nh = 10;
model_ten = Sequential();
model_ten.add(Dense(nh, input_shape=(nin,), activation="sigmoid", name="hidden"))
model_ten.add(Dense(nout, activation="sigmoid", name="output"))
# We will use 200 hidden units in our network here with sigmoid as activation
nh = 200; 
model_twohund = Sequential();
model_twohund.add(Dense(nh, input_shape=(nin,), activation="sigmoid", name="hidden"))
model_twohund.add(Dense(nout, activation="sigmoid", name="output"))


# We will have 3 hidden layers in this network with 100 units each, sigmoid as activation
nh = 100
model_twolayer = Sequential();
model_twolayer.add(Dense(nh, input_shape=(nin,), activation="sigmoid", name="hidden"))
model_twolayer.add(Dense(nh, input_shape=(nin,), activation="sigmoid", name="hidden_two"))
model_twolayer.add(Dense(nout, activation="sigmoid", name="output"))
    
# We will have 5 hidden layers in this network with 100 units each, sigmoid as activation
model_fivelayer = Sequential();
for x in range(0, 5):
    model_fivelayer.add(Dense(nh, input_shape=(nin,), activation="sigmoid", name="hidden" + str(x)))
model_fivelayer.add(Dense(nout, activation="sigmoid", name="output"))

modelArr = [model_sig, model_tanh, model_relu, model_ten, model_twohund, model_twolayer, 
           model_fivelayer];

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Check input layer, should be 8
print("Number of input nodes = %i" %nin)

Number of input nodes = 8


In [239]:
model_sig.summary()
model_twohund.summary()
model_fivelayer.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               900       
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 200)               1800      
_________________________________________________________________
output (Dense)               (None, 1)                 201       
Total params: 2,001
Trainable params: 2,001
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer 

In [240]:
from keras import optimizers

# Compile all models
opt = optimizers.Adam(lr=0.001)

for model in modelArr: 
    model.compile(optimizer=opt, 
             loss="binary_crossentropy",
             metrics=["accuracy"])
    
# model_sig.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])
# model_tanh.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])
# model_relu.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])
# model_ten.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])
# model_twohund.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])
# model_twolayer.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])
# model_fivelayer.compile(optimizer=opt, 
#              loss="binary_crossentropy",
#              metrics=["accuracy"])



In [241]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
plt.plot(model_relu.fit(x_train, y_train, epochs=10, batch_size=100, 
                        validation_data=(x_test, y_test), verbose=0))


In [231]:
model_relu.save("pulsar100_sigmoid");

In [232]:
from keras.models import load_model

model = load_model("pulsar100_sigmoid")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               900       
_________________________________________________________________
output (Dense)               (None, 1)                 101       
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


In [233]:
score, acc = model.evaluate(x_test, y_test, verbose = 0)
print("accuracy = %f" % acc)

accuracy = 0.943240
